<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/echo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to hammer Cytoscape by sending a long sequence of short operations. On its own, it's an effective test of Jupyter-Bridge -- it verifies that a Colab Notebook can keep a reliable channel to a private Cytoscape.

But when this test is executed in multiple Colabs (each connected to its own Cytoscape in its own VM), it verifies that Jupyter-Bridge can process a high volume of requests while keeping good track of requests and replies.


---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [1]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cytoscape/py4cytoscape (to revision 1.6.0) to /tmp/pip-req-build-6rlj9pcn


  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-6rlj9pcn
  Running command git checkout -b 1.6.0 --track origin/1.6.0
  Switched to a new branch '1.6.0'
  Branch '1.6.0' set up to track remote branch '1.6.0' from 'origin'.


  Created wheel for py4cytoscape: filename=py4cytoscape-1.6.0-py3-none-any.whl size=170883 sha256=d69b4eca7ba9eed2f33623e922eac8a21390597c3fd3df00184e9c904ca5de0b
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ps_37bu/wheels/41/f8/55/7d9713a08400a67188976c9214b7e48e0da9cd32b53ad30109
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=817e490d5278972b43675d3243f917b2a51161164e32636b60766683f8edde00
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built py4cytoscape python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [2]:
p4c.sandbox_set(None, copy_samples=False)

'/home/barry/CytoscapeConfiguration/filetransfer/default_sandbox'

In [3]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.6.0',
 'jupyterBridgeVersion': '0.0.2'}

## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [4]:
import uuid as uu
my_uuid = str(p4c.get_browser_client_channel())

print(f'Fixed message: {my_uuid}')


Fixed message: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961


## Test
Call Cytoscape's echo command with a unique message and get back the unique message.

The message consists of a random uuid (previously generated) combined with a monotonically increasing value.

Thus messages sent from this Colab instance are guaranteed to be unique in the universe, and each message sent from this Colab instance is guaranteed to be different from any previous messages.


In [5]:
%%time

import py4cytoscape as p4c

for i in range(0, 1000):
  message = f'{i}: {my_uuid}'
  ret = p4c.command_echo(message)
  if message == ret[0]:
    print(f'Successful: {message}')
  else:
    print(f'Failure: got {ret}, expected {message}')


Successful: 0: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 1: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 2: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 3: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 4: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 5: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 6: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 7: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 8: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 9: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 10: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 11: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 12: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 13: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 14: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 15: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 16: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 17: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Successful: 18: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961
Suc

In [6]:
res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")

def load_network():
  p4c.close_session(False)
  try:
    p4c.import_network_from_file("BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
  except:
    pass
  p4c.load_table_data_from_file("GDS112_full.soft", start_load_row=83, data_key_column_index=10, delimiters="\t")

In [7]:
%%time

for i in range(0, 10):
  load_network()
  print(f'Load network {i}: {my_uuid} successful')

In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 0: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 1: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


...

Load network 7: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 8: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 9: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
CPU times: user 3.88 s, sys: 427 ms, total: 4.3 s
Wall time: 9min 16s


In [8]:
load_network()
all_columns = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


In [9]:
%%time

for i in range(0, 20):
  column_vals = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])
  print(f'Get columns {i}: {my_uuid} {"successful" if all_columns.equals(column_vals) else "failed"}')



Get columns 0: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 1: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 2: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 3: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 4: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 5: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 6: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 7: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 8: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 9: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 10: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 11: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 12: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 13: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 14: 3256cb98-4dbf-4e9b-8b4c-0a2df2356961 successful
Get columns 15: 3256cb98-4dbf-4e9b-8b4c-0a2df23569